# Part 1

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import json

In [5]:
lines = None
with open('q1-test.json') as f:
    lines = f.readlines()
s=''
for i in lines:
    s += i
    
temp = json.loads(s)
observs = [None]*len(temp)
for idx, i in enumerate(temp):
    observs[idx] = [ord(j) - ord('a') for j in i]
for i in range(len(observs)):
    observs[i] = np.array(observs[i])

In [53]:
def normalize(arr):
    for i in range(len(arr)):
        arr[i] = arr[i] / np.sum(arr[i])
    return arr

def forward(A, B, pi, obs):
    T = len(obs)
    N = A.shape[0]
    k = B.shape[1]
    
    normalized = []
    alpha = np.zeros((N, T))
    alpha[:, 0] = pi * B[:, obs[0]]
    normalized.append(np.sum(alpha[:, 0]))
    alpha[:, 0] /= np.sum(alpha[:, 0] + 1e-9)
    for j in range(1, T):
        for i in range(N):
            alpha[i, j] = B[i, obs[j]] * (alpha[:, j - 1] @ A[:, i])
        normalized.append(np.sum(alpha[:, j]))
        alpha[:, j] /= np.sum(alpha[:, j] + 1e-9)
    
    return alpha, normalized

def backward(A, B, pi, obs, normalizer=None):
    T = len(obs)
    N = A.shape[0]
    k = B.shape[1]
    
    beta = np.zeros((N, T))
    for i in range(N):
        beta[i, T - 1] = 1
    beta[:, T - 1] *= normalizer[T - 1]
    
    for i in range(1, T):
        j = T - i - 1
        beta[:, j] = A @ (beta[:, j + 1] * B[:, obs[j + 1]])
        if normalizer != None:
            beta[:, j] *= normalizer[j]
    return beta
    
    
def new_matrices(A, B, pi, obs):
    T = len(obs)
    N = A.shape[0]
    k = B.shape[1]
    alpha, normalizer = forward(A, B, pi, obs)
    beta = backward(A, B, pi, obs, normalizer)
    
    gamma = np.zeros((N, T))
    
    for j in range(1, T):
        for i in range(N):
            gamma[i, j] = (alpha[i, j] * beta[i, j])/(alpha[:, j] @ beta[:, j])
            
    psi = np.zeros((N, N, T - 1))
    for i in range(N):
        for j in range(N):
            for k in range(T - 1):
                psi[i, j, k] = (alpha[i, k] * A[i, j] * beta[j, k + 1] * B[j, obs[k + 1]])/(alpha[:, k] @ (A @ (beta[:, k + 1] * B[:, obs[k + 1]])))
    k = B.shape[1]
    new_pi = gamma[:, 0]
    new_A = sum([psi[:, :, i] for i in range(T - 1)]) / sum([gamma[:, i] for i in range(T - 1)])
    new_B = np.zeros((N, k))
    for i in range(N):
        for j in range(k):
            new_B[i, j] = np.sum([i for idx, i in enumerate(gamma[i]) if obs[idx] == j])/np.sum(gamma[i])
    return normalize(new_A), normalize(new_B), new_pi / (np.sum(new_pi) + 1e-9)

In [54]:


A = normalize(np.random.uniform(size=(5, 5)))
B = normalize(np.random.uniform(size=(5, 7)))
pi = np.random.uniform(size=(5))
pi /= np.sum(pi)



new_matrices(A, B, pi, observs[0])

def baum_welch_multiple(A, B, pi, observs, n=5):
    
    for k in range(n):
        new_A = np.zeros(shape=A.shape)
        new_B = np.zeros(shape=B.shape)
        new_pi = np.zeros(shape=pi.shape)
        for i in observs:
            temp1, temp2, temp3 = new_matrices(A, B, pi, i)
            new_A += temp1
            new_B += temp2
            new_pi += temp3

        new_A /= len(observs)
        new_B /= len(observs)
        new_pi /= len(observs)

        A = normalize(new_A)
        B = normalize(new_B)
        pi = new_pi
        pi /= np.sum(new_pi + 1e-9)
    return A, B, pi
baum_welch_multiple(A, B, pi, observs)
        


<ipython-input-53-5ca58c2c1eb6>:53: RuntimeWarning: invalid value encountered in double_scalars
  gamma[i, j] = (alpha[i, j] * beta[i, j])/(alpha[:, j] @ beta[:, j])
<ipython-input-53-5ca58c2c1eb6>:59: RuntimeWarning: invalid value encountered in double_scalars
  psi[i, j, k] = (alpha[i, k] * A[i, j] * beta[j, k + 1] * B[j, obs[k + 1]])/(alpha[:, k] @ (A @ (beta[:, k + 1] * B[:, obs[k + 1]])))


(array([[nan, nan, nan, nan, nan],
        [nan, nan, nan, nan, nan],
        [nan, nan, nan, nan, nan],
        [nan, nan, nan, nan, nan],
        [nan, nan, nan, nan, nan]]),
 array([[nan, nan, nan, nan, nan, nan, nan],
        [nan, nan, nan, nan, nan, nan, nan],
        [nan, nan, nan, nan, nan, nan, nan],
        [nan, nan, nan, nan, nan, nan, nan],
        [nan, nan, nan, nan, nan, nan, nan]]),
 array([0., 0., 0., 0., 0.]))

In [ ]:
def forward_temp(A, B, pi, obs):
    T = len(obs)
    N = A.shape[0]
    k = B.shape[1]
    
    alpha = np.zeros((N, T))
    alpha[:, 0] = pi * B[:, obs[0]]
    alpha[:, 0] /= np.sum(alpha[:, 0])
    for j in range(1, T):
        for i in range(N):
            alpha[i, j] = B[i, obs[j]] * (alpha[:, j - 1] @ A[:, i])

    return alpha

def obs_prob(A, B, pi, obs):
    beta = backward(A, B, pi, obs)
    return np.dot(pi, beta[:, ])

lines = None
with open('q1-test.json') as f:
    lines = f.readlines()
s=''
for i in lines:
    s += i
    
temp = json.loads(s)
observs_test = [None]*len(temp)
for idx, i in enumerate(temp):
    observs_test[idx] = [ord(j) - ord('a') for j in i]
for i in range(len(observs)):
    observs_test[i] = np.array(observs_test[i])
    
for idx, i in enumerate(observs_test):
    print(f'Accuracy {idx} : {obs_prob(A, B, pi, i)}')

# Part 2

In [5]:
num_states = 5
num_obs = 7

T = np.zeros((num_states, num_states))
O = np.zeros((num_states, num_obs))
PI = np.zeros((num_states))

lines = None
with open('q2-train.json') as f:
    lines = f.readlines()
s=''
for i in lines:
    s += i
    
temp = json.loads(s)
observs2 = [None]*len(temp)
for idx, i in enumerate(temp):
    observs2[idx] = [ord(j) - ord('a') for j in i]
for i in range(len(observs2)):
    observs2[i] = np.array(observs2[i])
    
    
lines = None
with open('q2-train-hidden.json') as f:
    lines = f.readlines()
s=''
for i in lines:
    s += i
    
temp = json.loads(s)
hiddens = [None]*len(temp)
for idx, i in enumerate(temp):
    hiddens[idx] = [ord(j) - ord('0') for j in i]
for i in range(len(hiddens)):
    hiddens[i] = np.array(hiddens[i])

In [16]:
def normalize(arr):
    for i in range(len(arr)):
        arr[i] = arr[i] / np.sum(arr[i])
    return arr

for i in hiddens:
    PI[i] += i[0]
    
for i in hiddens:
    for j in range(1, len(i)):
        T[i[j - 1], i[j]] += 1

for i in range(len(hiddens)):
    if len(observs2[i]) != len(hiddens[i]):
        raise Exception()
    for j in range(len(hiddens[i])):
        O[hiddens[i][j], observs2[i][j]] += 1

T = normalize(T)
O = normalize(O)
PI = PI / np.sum(PI)

T

array([[0.20055469, 0.        , 0.39957168, 0.39987364, 0.        ],
       [0.        , 0.39983539, 0.20082606, 0.39933855, 0.        ],
       [0.39963727, 0.30025521, 0.        , 0.        , 0.30010752],
       [0.40028119, 0.30015197, 0.        , 0.        , 0.29956684],
       [0.        , 0.        , 0.19946893, 0.40068062, 0.39985045]])

In [8]:
def viterbi(T, O, PI, obs):
    dp = np.zeros((num_states, len(obs)))
    recurse = np.zeros((num_states, len(obs)), dtype=np.int8)
    for i in range(num_states):
        dp[i, 0] = PI[i] * O[i, obs[0]]
        recurse[i, 0] = -1
    
    for j in range(1, len(obs)):
        for i in range(num_states):
            arr = [dp[k, j - 1] * T[k, i] * O[i, obs[j]] for k in range(num_states)]
            recurse[i, j] = np.argmax(arr)
            dp[i, j] = np.max(arr)
        dp[:, j] *= 3
    
    idx = len(obs) - 1
    s = np.argmax(dp[:, idx])
    states = [s]
    while idx >= 0:
        s = recurse[s, idx]
        idx -= 1
        states.append(s)
    return states[:-1][::-1]


In [9]:
lines = None
with open('q2-test.json') as f:
    lines = f.readlines()
s=''
for i in lines:
    s += i
    
temp = json.loads(s)
observs_test = [None]*len(temp)
for idx, i in enumerate(temp):
    observs_test[idx] = [ord(j) - ord('a') for j in i]
for i in range(len(observs_test)):
    observs_test[i] = np.array(observs_test[i])

In [10]:
ans = viterbi(T, O, PI, observs_test[0])

In [11]:
for i in range(len(ans)):
    print(f'{ans[i]}->{chr(observs_test[0][i] + ord("a"))}')

0->b
3->c
0->a
3->c
0->a
2->e
0->b
2->g
0->b
3->c
1->d
1->g
2->f
0->b
2->e
1->d
3->c
0->b
0->d
2->f
0->a
2->d
0->b
0->c
3->c
0->b
2->d
0->a
3->c
0->c
2->g
1->e
1->d
3->c
4->g
4->a
4->g
3->c
4->g
3->c
4->c
3->c
1->d
2->g
4->e
3->c
4->g
3->c
0->a
3->c
4->f
4->b
3->c
0->g
2->g
4->f
4->g
2->e
0->b
0->b
3->c
0->a
2->e
0->b
3->c
0->a
2->d
0->a
3->c
0->b
2->d
4->g
3->c
4->g
3->c
4->b
4->e
3->c
0->a
3->c
4->g
3->c
4->g
4->f
3->c
1->d
1->d
1->e
1->d
3->c
0->a
2->e
1->d
3->c
0->a
3->c
1->d
1->d
2->g
0->b


In [13]:
T

array([[0.20055469, 0.        , 0.39957168, 0.39987364, 0.        ],
       [0.        , 0.39983539, 0.20082606, 0.39933855, 0.        ],
       [0.39963727, 0.30025521, 0.        , 0.        , 0.30010752],
       [0.40028119, 0.30015197, 0.        , 0.        , 0.29956684],
       [0.        , 0.        , 0.19946893, 0.40068062, 0.39985045]])

In [14]:
O

array([[0.19960155, 0.5010026 , 0.09988979, 0.09961925, 0.        ,
        0.        , 0.09988681],
       [0.09978744, 0.10052268, 0.09960513, 0.40009736, 0.09975839,
        0.10024622, 0.09998277],
       [0.        , 0.        , 0.        , 0.25075263, 0.24979326,
        0.24957091, 0.2498832 ],
       [0.        , 0.        , 1.        , 0.        , 0.        ,
        0.        , 0.        ],
       [0.150348  , 0.15016344, 0.14049832, 0.13974605, 0.13978116,
        0.13970694, 0.13975609]])

In [21]:
PI

array([0.5986, 0.0989, 0.1029, 0.1   , 0.0996])

In [20]:
PI = np.zeros((num_states))
for i in hiddens:
    PI[i[0]] += 1
PI = PI / np.sum(PI)